**Data prep**

**2022: Week 6 - 7 letter Scrabble Words**

<a href = "https://preppindata.blogspot.com/2022/02/2022-week-6-7-letter-scrabble-words.html" >Data source and requirements </a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import re as re
import math
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.14f' % x)

**Input the excel file**

In [2]:
data= pd.read_excel("7 letter words.xlsx",sheet_name = ["7 letter words","Scrabble Scores","Scaffold"])
data.keys()

dict_keys(['7 letter words', 'Scrabble Scores', 'Scaffold'])

In [3]:
df_ScrabbleScores=data['Scrabble Scores']
df_7letterwords=data['7 letter words']
df_Scaffold=data['Scaffold']

**Parse out the information in the Scrabble Scores Input so that there are 3 fields:**
- Tile
- Frequency
- Points

In [4]:
df_ScrabbleScores[['P1','P2']] = df_ScrabbleScores.Scrabble.str.split(":",expand=True)
df_ScrabbleScores

,Scrabble,Unnamed: 1,P1,P2
0,0 points: Blank ×2,NaN,0 points,Blank ×2
1,"1 point: E ×12, A ×9, I ×9, O ×8, N ×6, R ×6, ...",NaN,1 point,"E ×12, A ×9, I ×9, O ×8, N ×6, R ×6, T ×6, L ..."
2,"2 points: D ×4, G ×3",NaN,2 points,"D ×4, G ×3"
3,"3 points: B ×2, C ×2, M ×2, P ×2",NaN,3 points,"B ×2, C ×2, M ×2, P ×2"
4,"4 points: F ×2, H ×2, V ×2, W ×2, Y ×2",NaN,4 points,"F ×2, H ×2, V ×2, W ×2, Y ×2"
5,5 points: K ×1,NaN,5 points,K ×1
6,"8 points: J ×1, X ×1",NaN,8 points,"J ×1, X ×1"
7,"10 points: Q ×1, Z ×1",NaN,10 points,"Q ×1, Z ×1"


In [5]:
#Create a new dataframe from the series with P1 as the index
new_df = pd.DataFrame(df_ScrabbleScores.P2.str.split(',').tolist(), index=df_ScrabbleScores.P1).stack()

#We now want to get rid of the secondary index
# To do this, we will make P1 as a column (it can't be an index since the values will be duplicate)
new_df = new_df.reset_index([0, 'P1'])

#set the column names
new_df.columns = ['Pts', 'TF']


In [6]:
new_df[['Tile','F1']] = new_df.TF.str.split(expand=True)

#Extract only number from the specified column
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)
new_df['Frequency']=new_df['F1'].apply(lambda x: find_number(x))

new_df['Points']=new_df.Pts.str.split(" ",expand=True)[0]
new_df

#output desired columns
output_columns = ["Tile", "Frequency", "Points"]
df_ScrabbleScores = new_df[output_columns]

df_ScrabbleScores['Frequency']=df_ScrabbleScores['Frequency'].astype(int)
df_ScrabbleScores['Points']=df_ScrabbleScores['Points'].astype(int)

**Calculate the % Chance of drawing a particular tile and round to 2 decimal places**
- Frequency / Total number of tiles

In [7]:
df_ScrabbleScores["% Chance"] = (df_ScrabbleScores['Frequency'].astype(int)/(df_ScrabbleScores['Frequency'].astype(int).sum()).round(2))

df_ScrabbleScores


,Tile,Frequency,Points,% Chance
0,Blank,2,0,0.02000000000000
1,E,12,1,0.12000000000000
2,A,9,1,0.09000000000000
3,I,9,1,0.09000000000000
4,O,8,1,0.08000000000000
5,N,6,1,0.06000000000000
6,R,6,1,0.06000000000000
7,T,6,1,0.06000000000000
8,L,4,1,0.04000000000000
9,S,4,1,0.04000000000000


**Split each of the 7 letter words into individual letters and count the number of occurrences of each letter**



In [8]:
df_7letterwords['7 letter word'] = df_7letterwords['7 letter word'].str.lower()
df_7letterwords['F']=df_7letterwords['7 letter word'].apply(len)
print("Number of times: " , df_7letterwords['F'].value_counts())


Number of times:  7    973
Name: F, dtype: int64


In [9]:
nos = [0,1,2,3,4,5,6]
words = list(df_7letterwords['7 letter word'])

no = np.tile(nos, len(words))
word = np.repeat(words, len(nos))
df = pd.DataFrame({"number": no, "word": word})

df['Character'] = df.apply(lambda x: x['word'][x['number']],1) 

occur = df.groupby(['word', 'Character']).size().reset_index()

occur.columns=['Word','Letter','Number of Occurrences']
occur

,Word,Letter,Number of Occurrences
0,abbozzo,a,1
1,abbozzo,b,2
2,abbozzo,o,2
3,abbozzo,z,2
4,ability,a,1
...,...,...,...
5897,zythums,z,1
5898,zyzzyva,a,1
5899,zyzzyva,v,1
5900,zyzzyva,y,2


**Join each letter to its scrabble tile**

In [10]:
df_ScrabbleScores['Tile'] = df_ScrabbleScores['Tile'].str.lower()
occur['Letter'] = occur['Letter'].str.lower()
data=pd.merge(df_ScrabbleScores,occur, left_on='Tile', right_on='Letter')
data

,Tile,Frequency,Points,% Chance,Word,Letter,Number of Occurrences
0,e,12,1,0.12000000000000,absence,e,2
1,e,12,1,0.12000000000000,academy,e,1
2,e,12,1,0.12000000000000,accused,e,1
3,e,12,1,0.12000000000000,achieve,e,2
4,e,12,1,0.12000000000000,acquire,e,1
...,...,...,...,...,...,...,...
5897,z,1,10,0.01000000000000,zymomes,z,1
5898,z,1,10,0.01000000000000,zymotic,z,1
5899,z,1,10,0.01000000000000,zymurgy,z,1
5900,z,1,10,0.01000000000000,zythums,z,1


**Update the % chance of drawing a tile based on the number of occurrences in that word**
- If the word contains more occurrences of that letter than the frequency of the tile, set the probability to 0 - it is impossible to make this word in Scrabble
- Remember for independent events, you multiple together probabilities i.e. if a letter appears more than once in a word, you will need to multiple the % chance by itself that many times


In [11]:
l=[]
for index, row in data.iterrows():
    if row['Number of Occurrences']<=row['Frequency']:
        l.append(math.pow(row['% Chance'],row['Number of Occurrences']))
    else:
        l.append(0)        

In [12]:
data['% Chance']=l

**Calculate the total points each word would score**

In [13]:
data

,Tile,Frequency,Points,% Chance,Word,Letter,Number of Occurrences
0,e,12,1,0.01440000000000,absence,e,2
1,e,12,1,0.12000000000000,academy,e,1
2,e,12,1,0.12000000000000,accused,e,1
3,e,12,1,0.01440000000000,achieve,e,2
4,e,12,1,0.12000000000000,acquire,e,1
...,...,...,...,...,...,...,...
5897,z,1,10,0.01000000000000,zymomes,z,1
5898,z,1,10,0.01000000000000,zymotic,z,1
5899,z,1,10,0.01000000000000,zymurgy,z,1
5900,z,1,10,0.01000000000000,zythums,z,1


In [14]:
data['Points']=data['Points']*data['Number of Occurrences']

In [15]:
dftp=pd.DataFrame(data.groupby(["Word"])['Points'].sum().reset_index())
dftp.columns=["Word",'Total Points']


data1=pd.merge(data,dftp, left_on='Word', right_on='Word')


output_columns = ["Word","Total Points","Letter","% Chance"]
data = data1[output_columns]


In [16]:
new_data=pd.pivot_table(data, values='% Chance', index=['Word', 'Total Points'],
                    columns=['Letter'], aggfunc=np.sum).reset_index()


In [17]:
new_data['a'] = np.where(new_data['a'].isnull(), 1, new_data['a'] )
new_data['b'] = np.where(new_data['b'].isnull(), 1, new_data['b'] )
new_data['c'] = np.where(new_data['c'].isnull(), 1, new_data['c'] )
new_data['d'] = np.where(new_data['d'].isnull(), 1, new_data['d'] )
new_data['e'] = np.where(new_data['e'].isnull(), 1, new_data['e'] )
new_data['f'] = np.where(new_data['f'].isnull(), 1, new_data['f'] )
new_data['g'] = np.where(new_data['g'].isnull(), 1, new_data['g'] )
new_data['h'] = np.where(new_data['h'].isnull(), 1, new_data['h'] )
new_data['i'] = np.where(new_data['i'].isnull(), 1, new_data['i'] )
new_data['j'] = np.where(new_data['j'].isnull(), 1, new_data['j'] )
new_data['k'] = np.where(new_data['k'].isnull(), 1, new_data['k'] )
new_data['l'] = np.where(new_data['l'].isnull(), 1, new_data['l'] )
new_data['m'] = np.where(new_data['m'].isnull(), 1, new_data['m'] )
new_data['n'] = np.where(new_data['n'].isnull(), 1, new_data['n'] )
new_data['o'] = np.where(new_data['o'].isnull(), 1, new_data['o'] )
new_data['p'] = np.where(new_data['p'].isnull(), 1, new_data['p'] )
new_data['q'] = np.where(new_data['q'].isnull(), 1, new_data['q'] )
new_data['r'] = np.where(new_data['r'].isnull(), 1, new_data['r'] )
new_data['s'] = np.where(new_data['s'].isnull(), 1, new_data['s'] )
new_data['t'] = np.where(new_data['t'].isnull(), 1, new_data['t'] )
new_data['u'] = np.where(new_data['u'].isnull(), 1, new_data['u'] )
new_data['v'] = np.where(new_data['v'].isnull(), 1, new_data['v'] )
new_data['w'] = np.where(new_data['w'].isnull(), 1, new_data['w'] )
new_data['x'] = np.where(new_data['x'].isnull(), 1, new_data['x'] )
new_data['y'] = np.where(new_data['y'].isnull(), 1, new_data['y'] )
new_data['z'] = np.where(new_data['z'].isnull(), 1, new_data['z'] )


In [18]:
new_data['Total % chance'] = new_data['a'] * new_data['b'] * new_data['c'] * new_data['d'] * new_data['e'] * new_data['f']* new_data['g'] * new_data['h'] * new_data['i'] * new_data['j'] * new_data['k']* new_data['l'] * new_data['m'] * new_data['n'] * new_data['o'] * new_data['p']* new_data['q'] * new_data['r'] * new_data['s'] * new_data['t']* new_data['u']* new_data['v'] * new_data['w'] * new_data['x'] * new_data['y'] * new_data['z']

                           

In [19]:
output_columns = ["Word","Total % chance","Total Points"]
new_data = new_data[output_columns]

**Filter out words with a 0% chance**

In [20]:
new_data=new_data[new_data['Total % chance']!=0.00000000000000]

**Rank the words by their % chance (dense rank)**

In [21]:
new_data['Likelihood Rank'] = new_data['Total % chance'].rank(ascending=0,method='dense').astype(int)

**Rank the words by their total points (dense rank)**

In [22]:
new_data['Points Rank'] = new_data['Total Points'].rank(ascending=0,method='dense').astype(int)

In [23]:
new_data.sort_values(by=['Points Rank'], inplace=True)

In [24]:
new_data

Letter,Word,Total % chance,Total Points,Likelihood Rank,Points Rank
548,muzjiks,0.00000000000288,29,178,1
442,jukebox,0.00000000000768,27,172,2
66,bezique,0.00000000010368,27,121,2
524,mezquit,0.00000000005184,27,138,2
864,tzaddiq,0.00000000007776,27,129,2
...,...,...,...,...,...
557,natural,0.00000000279936,7,26,22
781,session,0.00000000331776,7,22,22
34,assault,0.00000000124416,7,48,22
841,sustain,0.00000000186624,7,36,22


In [25]:
output_columns = ["Points Rank","Likelihood Rank","Word","Total % chance","Total Points" ]
df = new_data[output_columns]

# Output the data 
df.to_csv('PD 2022 Week 6 Output.csv', index=False)



In [26]:
print("Data Prepped!")

Data Prepped!
